In [1]:
import pandas as pd
import numpy as np
import networkx as nx

### Find number of neighbours & Node Degree

In [ ]:
# find neibhours for a given node

In [2]:
fb_network = pd.read_csv("fb_network_sub.csv")

In [4]:
fb_network.head()

,source_node,destination_node
0,741,279438
1,741,1858237
2,741,321689
3,741,74643
4,741,1517039


In [8]:
G = nx.from_pandas_edgelist(fb_network, source="source_node", target="destination_node")


In [9]:
# how many friends does user 1 has
len(list(G.neighbors(741)))

371

In [10]:
# same as 
val1 = fb_network.loc[fb_network["source_node"] == 741,"destination_node"].values
val2 = fb_network.loc[fb_network["destination_node"] == 741, "source_node"].values

In [11]:
len(set(list(np.append(val1,val2))))

371

<b> Find nodes with influence </b>

In [21]:
## Node centrality
fb_deg_cen = nx.degree_centrality(G)

In [13]:
type(fb_deg_cen)

dict

In [14]:
fb_deg_cen[741]

0.003965327433439147

In [15]:
# same as 
len(list(G.neighbors(741)))/(len(list(G.nodes())) - 1)

0.003965327433439147

### Link Prediction (or Recommendation)

<b> Common Neighbors Method </b>

In [16]:
## common neighbours
## find common neighbours for a paticular user say user no 4850
len(list(G.neighbors(4850)))

896

In [17]:
# find list of key users the user 4850 is not currently connected to
key_users = list(set(fb_network["source_node"].values))
unconnected_users = [x for x in key_users if x not in list(G.neighbors(4850))]

In [18]:
len(unconnected_users)

2892

In [33]:
len(list(nx.common_neighbors(G,4850,1492489)))

186

In [19]:
# find common neighbours
common_neighbors = []
for user_id in unconnected_users:
    common_neighbors.append(len(list(nx.common_neighbors(G, 4850, user_id))))

In [22]:
common_neighbors_df = pd.DataFrame({"user_id": 4850, 
                                    "unconnctd_user_id": unconnected_users, "common_neighbors": common_neighbors})

In [23]:
common_neighbors_df.sort_values(by = "common_neighbors", ascending=False).head()

,user_id,unconnctd_user_id,common_neighbors
1743,4850,4850,896
544,4850,1492489,186
122,4850,1622389,57
1482,4850,1855498,42
2210,4850,1079335,42


In [24]:
n1 = set(G.neighbors(4850))
n2 = set(G.neighbors(1492489))

In [25]:
len(n1.intersection(n2))

186

<b> Using Jaccard Similarity </b>

In [37]:
list(nx.jaccard_coefficient(G, [(4850,1492489)]))

[(4850, 1492489, 0.08172231985940245)]

In [27]:
len(n1.intersection(n2))/len(n1.union(n2))

0.08172231985940245

In [28]:
jac_sim = [list(nx.jaccard_coefficient(G, [(4850,user_id)]))[0] for user_id in unconnected_users]

In [29]:
jac_sim_df =  pd.DataFrame.from_records(jac_sim,columns=["user_id","unconnctd_user_id","jac_sim"])

In [30]:
jac_sim_df.sort_values(by="jac_sim", ascending=False).head()

,user_id,unconnctd_user_id,jac_sim
1743,4850,4850,1.000000
544,4850,1492489,0.081722
122,4850,1622389,0.051167
2858,4850,245648,0.038986
577,4850,1123930,0.036530


In [31]:
n1 = set(G.neighbors(4850))
n2 = set(G.neighbors(245648))

In [32]:
print("intersection -" ,len(n1.intersection(n2)))
print("union -" , len(n1.union(n2)))

intersection - 40
union - 1026
